<a href="https://colab.research.google.com/github/Tilli-simgame/SG-CompetitionNators/blob/main/estekisojen_muotoonlaittonaattori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kilpailutulosten hakutyökalu

Tämä työkalu hakee kilpailutuloksia annetun ratsastajan ja VRL-numeron perusteella.

## Käyttöohjeet:
1. Aja solut järjestyksessä ylhäältä alas (Shift + Enter tai play-nappi solun vasemmalla puolella)
2. Ensimmäinen solu asentaa tarvittavat kirjastot
3. Syötä kilpailujen URL-osoitteet ja haettavan ratsastajan tiedot niille varattuihin soluihin
4. Aja viimeinen solu tulosten hakemiseksi

## Huomioitavaa:
- URL-osoitteiden tulee olla url osoitteita, sisällytä siis myös "http(s)://" osuus.
- Ratsastajan nimen ja VRL-numeron tulee olla täsmälleen oikeassa muodossa (esim. "Tilli (VRL-00406)")

# 1. Kirjastojen asennus

Suorita tämä solu ensimmäisenä. Se asentaa tarvittavat Python-kirjastot.

In [ ]:
!pip install requests beautifulsoup4 pandas

# 2. Python-kirjastojen tuonti

Tämä solu lataa tarvittavat työkalut käyttöön.

In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

# 3. Hakutoiminnot

Tämä solu sisältää ohjelman toiminnallisuuden. Suorita solu, mutta koodia ei tarvitse muokata.

In [16]:
def clean_html(text):
    text = re.sub(r'</?strong>', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_event_details(text):
    date_patterns = [
        r'Kilpailupäivä\s*\*\*([^*]+?)\*\*',
        r'Kilpailupäivä\s*([0-9]{1,2}\.[0-9]{1,2}\s*[0-9]{4})',
        r'Kilpailupäivä\s+([^\n,]+)',
    ]

    competition_date = ""
    for pattern in date_patterns:
        date_match = re.search(pattern, text, re.IGNORECASE)
        if date_match:
            competition_date = date_match.group(1).strip()
            break

    location_patterns = [
        r'Järjestyspaikka\s*\*\*([^*]+?)\*\*',
        r'Järjestyspaikka\s+([^\n,]+)',
        r'Järjestyspaikka\s*([^,\n]+)'
    ]

    location = ""
    for pattern in location_patterns:
        location_match = re.search(pattern, text, re.IGNORECASE)
        if location_match:
            location = location_match.group(1).strip()
            location = location.replace('*', '').strip()
            break

    return competition_date, location

def extract_class_participants(text):
    class_participants = {}
    lines = text.split('\n')

    for line in lines:
        line = clean_html(line)
        if not line:
            continue

        class_match = re.match(r'(\d+)\.\s+(?:Puomiluokka|\d+\s*cm)\s+(\d+)/50', line)
        if class_match:
            class_num = class_match.group(1)
            participants = int(class_match.group(2))
            class_participants[class_num] = participants

    return class_participants

def search_competition_results(content, search_term):
    try:
        soup = BeautifulSoup(content, 'html.parser')
        text_content = soup.get_text()

        competition_date, location = extract_event_details(text_content)
        class_participants = extract_class_participants(text_content)

        results = []
        current_class = None
        current_class_num = None
        in_results = False

        lines = [clean_html(line) for line in content.split('\n')]

        for line in lines:
            if not line:
                continue

            if line.lower() == 'tulokset':
                in_results = True
                continue

            if not in_results:
                continue

            class_match = re.match(r'(\d+)\.\s+((?:Puomiluokka|\d+\s*cm))', line)
            if class_match:
                current_class_num = class_match.group(1)
                current_class = class_match.group(2).strip()
                current_class = re.sub(r'(\d+)\s*cm', r'\1cm', current_class)
                continue

            if search_term in line:
                placement_match = re.match(r'(\d+)\.', line)
                placement = placement_match.group(1) if placement_match else ''

                result_match = re.search(rf'{re.escape(search_term)}\s*-\s*(.*?)\s*(VH\d+-\d+-\d+)', line)

                if result_match:
                    horse_name = result_match.group(1).strip()
                    vh_number = result_match.group(2).strip()

                    results.append({
                        'class': current_class,
                        'class_number': int(current_class_num),
                        'placement': int(placement) if placement else 999,
                        'total_participants': class_participants.get(current_class_num, 0),
                        'horse_name': horse_name,
                        'vh_number': vh_number,
                        'competition_date': competition_date,
                        'location': location
                    })

        return results

    except Exception as e:
        print(f"Error processing content: {str(e)}")
        return []

def display_results_by_horse(results, save_to_file=False):
    if not results:
        print("No results found.")
        return

    df = pd.DataFrame(results)
    df = df.sort_values(by=['class_number', 'placement'])
    grouped = df.groupby(['horse_name', 'vh_number'])

    for (horse_name, vh_number), group in grouped:
        if not horse_name:
            continue

        print(f"\n{horse_name} ({vh_number}):")

        sorted_group = group.sort_values(by=['class_number', 'placement'])

        for _, row in sorted_group.iterrows():
            if row['placement'] != 999:
                print(f"{row['competition_date']} - {row['location']} - {row['class']} - {row['placement']}/{row['total_participants']}")

    if save_to_file:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'competition_results_{timestamp}.csv'
        df.to_csv(filename, index=False)
        print(f"\nResults saved to {filename}")

def process_multiple_urls(urls, search_term):
    all_results = []

    for url in urls:
        try:
            print(f"\nProcessing {url}...")
            response = requests.get(url)
            response.encoding = 'utf-8'
            results = search_competition_results(response.text, search_term)
            if results:
                print(f"Found {len(results)} results")
                all_results.extend(results)
            else:
                print("No results found in this URL")
        except Exception as e:
            print(f"Error processing {url}: {str(e)}")

    return all_results

# 4. Syötetiedot

Muokkaa alla olevat tiedot:
- Lisää kilpailujen URL-osoitteet urls-listaan
- Kirjoita search_term-kohtaan haettavan ratsastajan nimi tai VRL-numero täsmälleen oikeassa muodossa

Esimerkki:
- URL: https://harakkasyndrooma.net/n/krj/361.html
- Ratsastajan tiedot: "Tilli (VRL-00406)"

In [21]:
# Lisää kilpailujen URL-osoitteet tähän, yksi per rivi
urls = [
    #"https://harakkasyndrooma.net/n/erj/401.html",
    "https://harakkasyndrooma.net/n/erj/361.html" # Poista # merkki rivin alusta ottaaksesi URL käyttöön
]

# Kirjoita ratsastajan nimi tai VRL-numero
search_term = "corbin. (VRL-06467)"  # Muuta tämä

# 5. Tulosten haku

Suorita tämä solu hakeaksesi tulokset.
- Tulokset näytetään ruudulla
- CSV-tiedosto tallennetaan automaattisesti

In [22]:
# Hae ja näytä tulokset
results = process_multiple_urls(urls, search_term)
display_results_by_horse(results, save_to_file=True)


Processing https://harakkasyndrooma.net/n/erj/361.html...
Found 38 results

Abydos G! (VH14-034-0118):
11.01 2025 - Nessinjärven kilpailukeskus - 160cm - 10/36
11.01 2025 - Nessinjärven kilpailukeskus - 160cm - 25/36

Carusos Valerie (VH05-029-8825):
11.01 2025 - Nessinjärven kilpailukeskus - 90cm - 41/50

Euphorio SWA (VH24-021-0192):
11.01 2025 - Nessinjärven kilpailukeskus - 150cm - 29/40
11.01 2025 - Nessinjärven kilpailukeskus - 150cm - 36/40

Fawnmor Cinnabar (VH22-023-0128):
11.01 2025 - Nessinjärven kilpailukeskus - 70cm - 7/50
11.01 2025 - Nessinjärven kilpailukeskus - 80cm - 17/50

Fiktion Velmu (VH24-018-0430):
11.01 2025 - Nessinjärven kilpailukeskus - 90cm - 50/50

Heinämäen Luukas (VH05-018-4989):
11.01 2025 - Nessinjärven kilpailukeskus - 70cm - 44/50
11.01 2025 - Nessinjärven kilpailukeskus - 80cm - 31/50

Heinämäen Tuisku (VH24-018-0433):
11.01 2025 - Nessinjärven kilpailukeskus - 70cm - 41/50
11.01 2025 - Nessinjärven kilpailukeskus - 80cm - 7/50

Hystereesi (VH24-01